In [33]:
from scipy import io
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly
import plotly.io as pio
import Plotters
import open3d as o3d

pio.renderers.default = "browser"
path = 'H:\My Drive\Roni\csv_mov'
mov = 'mov15'
files_to_load = [f'hull_data_{mov}',f'real_coords_{mov}','rot_mat','wing_normals']

# load data to dictionary
data_dict = {file_to_load : pd.read_csv(f'{path}\{file_to_load}.csv') for file_to_load in files_to_load}





In [34]:
# load body and wing data to dictionary
name_of_parts = ['body','wing_l','wing_r']
fly_hull = {name_of_part : data_dict[files_to_load[0]].query(f'{name_of_part} == 1')[['x','y','z']] for name_of_part in name_of_parts}
# load real coordinates
real_coord = {ax: data_dict[files_to_load[1]].query(f'{ax} == 1')['real_coords'].reset_index(drop = True) for ax in ['x','y','z']}

In [35]:
# rotate and save the hull in lab axes
rot_mat = data_dict['rot_mat'].to_numpy()
real_hull = {body_wing : np.dot(rot_mat,np.vstack([real_coord[ax][fly_hull[body_wing][ax]] for ax in ['x','y','z']])).T for body_wing in name_of_parts}

In [8]:

Plotters.plot_hull(real_hull,size = 3)



In [131]:
from scipy import io
import numpy as np
import open3d as o3d
from skimage import measure   


def get_3d_voxel_grid(indices):
    volume_shape = (max(indices[:,0])+2, max(indices[:,1])+2, max(indices[:,2])+2)
    voxels = np.zeros(volume_shape)
    for ix,iy,iz in indices:
        voxels[ix-1, iy-1, iz-1] = 1
    return voxels
    

# load body and wing data to dictionary
name_of_parts = ['body','wing_l','wing_r']
fly_hull = {name_of_part : data_dict[files_to_load[0]].query(f'{name_of_part} == 1')[['x','y','z']].to_numpy() for name_of_part in name_of_parts}
pcd = {name_of_part : o3d.geometry.PointCloud(o3d.utility.Vector3dVector(fly_hull[name_of_part])) for name_of_part in name_of_parts}
voxel_grid = get_3d_voxel_grid(np.vstack((fly_hull.values())))

vertices, faces,normals, values = measure.marching_cubes(voxel_grid,0,allow_degenerate=False,spacing=(1, 1, 1))  


mesh = o3d.geometry.TriangleMesh()
vertices = vertices.astype(int)
real_hull_in_vertex_order =  np.dot(rot_mat,np.vstack([real_coord[ax][vertices[:,idx]]] for idx,ax in enumerate(['x','y','z']))).T 

# Assign vertices, faces, and normals to the mesh
mesh.vertices = o3d.utility.Vector3dVector(real_hull_in_vertex_order)
mesh.triangles = o3d.utility.Vector3iVector(faces)
mesh.compute_vertex_normals()
vertex_normals = -np.asarray(mesh.vertex_normals)


o3d.visualization.draw_geometries([mesh], point_show_normal=True, mesh_show_wireframe=False)
o3d.io.write_triangle_mesh('wakk' + ".ply", mesh)





C:\Users\Roni\AppData\Local\Temp\ipykernel_1396\4243656320.py:19: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

C:\Users\Roni\AppData\Local\Temp\ipykernel_1396\4243656320.py:29: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



True

In [88]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 21 13:39:05 2023

@author: Roni
"""

from scipy import io
import numpy as np
import open3d as o3d


def get_3d_voxel_grid(indices):
    volume_shape = (max(indices[:,0])+2, max(indices[:,1])+2, max(indices[:,2])+2)
    voxels = np.zeros(volume_shape)
    for ix,iy,iz in indices:
        voxels[ix-1, iy-1, iz-1] = 1
        return voxels
    

# load body and wing data to dictionary
name_of_parts = ['body','wing_l','wing_r']
fly_hull = {name_of_part : data_dict[files_to_load[0]].query(f'{name_of_part} == 1')[['x','y','z']].to_numpy() for name_of_part in name_of_parts}
pcd = {name_of_part : o3d.geometry.PointCloud(o3d.utility.Vector3dVector(fly_hull[name_of_part])) for name_of_part in name_of_parts}
voxel_grid = get_3d_voxel_grid(np.vstack((fly_hull.values())))

wing_body = np.vstack((fly_hull.values()))
voxels_grid = (max(wing_body[:,0])+2, max(wing_body[:,1])+2, max(wing_body[:,2])+2)


vertices, faces,normals, values = measure.marching_cubes(voxels_grid,0,allow_degenerate=False,spacing=(1, 1, 1))  






# body = fly_hull['body']
# rwing= fly_hull['wing_r']
# lwing= fly_hull['wing_l']



# fly_lab = np.vstack((real_hull['body'],real_hull['wing_r'],real_hull['wing_l']))

def gen_pcd(indices,real_coord,rot_mat,color):


    hull_for_pcd =np.dot(rot_mat,np.vstack([real_coord[ax][indices[:,idx]]] for idx,ax in enumerate(['x','y','z']))).T 

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(hull_for_pcd)
    return pcd


# pcd_all= gen_pcd(body,real_coord,rot_mat,color)

# pcd_body = gen_pcd(body,real_coord,rot_mat,np.vstack((np.tile([0.0,1.0,0.0],(body.shape[0],1)))))
# pcd_rwing= gen_pcd(rwing,real_coord,rot_mat,np.vstack((np.tile([0.0,1.0,0.0],(body.shape[0],1)))))
# pcd_lwing= gen_pcd(lwing,real_coord,rot_mat,np.vstack((np.tile([0.0,1.0,0.0],(body.shape[0],1)))))




from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay

# if not isinstance(rwing,Delaunay):


#     hull = Delaunay(rwing)
#     inner = rwing[hull.find_simplex(rwing)>=0]
#     outer = ConvexHull(inner)
#     outer = outer.points.astype(int)
    
# if not isinstance(rwing,Delaunay):
#     hull = Delaunay(lwing)
#     inner = lwing[hull.find_simplex(lwing)>=0]
#     outerl = ConvexHull(inner)
#     outerl = outerl.points.astype(int)

out_and_in = np.vstack((rwing,lwing))




pcd_all= gen_pcd(out_and_in,real_coord,rot_mat,out_and_in_color)

pcd_rw= gen_pcd(np.vstack((rwing)),real_coord,rot_mat,color)
pcd_lw= gen_pcd(np.vstack((lwing)),real_coord,rot_mat,color2)




from skimage import measure   
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection


wing_body = np.vstack((body,out_and_in))
volume_shape = (max(wing_body[:,0])+2, max(wing_body[:,1])+2, max(wing_body[:,2])+2)



voxels = np.zeros(volume_shape)
for ix,iy,iz in body:
    voxels[ix-1, iy-1, iz-1] = 1
    
for ix,iy,iz in out_and_in:
    voxels[ix-1, iy-1, iz-1] = 1
    
    
vertices, faces,normals, values = measure.marching_cubes(voxels,0,allow_degenerate=False,spacing=(1, 1, 1))  
mesh_body = o3d.geometry.TriangleMesh()


vertices = vertices.astype(int)

# body_lab = hull_to_lab_params['rotation_mat'].dot(np.array([hull_to_lab_params['real_x'][vertices[:,0]],hull_to_lab_params['real_y'][vertices[:,1]],hull_to_lab_params['real_z'][vertices[:,2]]])).T
hull_for_pcd =np.dot(rot_mat,np.vstack([real_coord[ax][vertices[:,idx]]] for idx,ax in enumerate(['x','y','z']))).T 

# Assign vertices, faces, and normals to the mesh
mesh_body.vertices = o3d.utility.Vector3dVector(hull_for_pcd.astype(float))
mesh_body.triangles = o3d.utility.Vector3iVector(faces)
mesh_body.compute_vertex_normals()
vertex_normals = -np.asarray(mesh_body.vertex_normals)



o3d.visualization.draw_geometries([mesh_body])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(mesh_body.vertices)
pcd.normals = o3d.utility.Vector3dVector(vertex_normals)


dists = pcd.compute_point_cloud_distance(pcd_rw)
dists = np.asarray(dists)
ind = np.where(dists <0.0001)[0]


dists = pcd.compute_point_cloud_distance(pcd_lw)
dists = np.asarray(dists)
indl = np.where(dists <0.0001)[0]


color = np.vstack((np.tile([0.0,1.0,0.0],(vertices.shape[0],1))))
color[ind] = [1,0,0]
color[indl] = [0,0,1]

pcd.colors = o3d.utility.Vector3dVector(color)

# =============================================================================
# pcd_rw += pcd
# 
# o3d.visualization.draw_geometries([pcd_rw], mesh_show_wireframe=True)
# =============================================================================



o3d.visualization.draw_geometries([pcd], mesh_show_wireframe=True)
o3d.visualization.draw_geometries([mesh_body], point_show_normal=True, mesh_show_wireframe=True)


o3d.io.write_triangle_mesh(filename + ".ply", mesh_body)

poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=11)[0]
o3d.visualization.draw_geometries([poisson_mesh], mesh_show_wireframe=True)
o3d.io.write_triangle_mesh(filename + "_poisson.ply", mesh_body)





C:\Users\Roni\AppData\Local\Temp\ipykernel_1396\220342885.py:25: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

C:\Users\Roni\AppData\Local\Temp\ipykernel_1396\220342885.py:27: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

C:\Users\Roni\AppData\Local\Temp\ipykernel_1396\220342885.py:46: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

C:\Users\Roni\AppData\Local\Temp\ipykernel_1396\220342885.py:46: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. S

True